DATA PRE-PROCESSING

In [12]:
import cv2
import numpy as np
import os

data_dir = "C:/Users/sachi/Documents/aisign/venv/dataset"
image_size = (64, 64)
x, y = [], []

for label, sign in enumerate(os.listdir(data_dir)):
    for img_name in os.listdir(os.path.join(data_dir, sign)):
        img_path = os.path.join(data_dir, sign, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, image_size)
        x.append(img)
        y.append(label)

x = np.array(x) / 255.0  # Normalize
y = np.array(y)


HAND LANDMARKS

In [13]:
import mediapipe as mp

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

def extract_hand_landmarks(image):
    results = hands.process(image)
    keypoints = []
    if results.multi_hand_landmarks:
        for hand in results.multi_hand_landmarks:
            for point in hand.landmark:
                keypoints.extend(point.x)
                keypoints.extend(point.y)
    
    return keypoints


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(29, activation='softmax')  
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x, y, epochs=10, validation_split=0.2)
model.save("ai_sign_model.h5")

c:\Users\sachi\Documents\aisign\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 79s 34ms/step - accuracy: 0.5852 - loss: 1.3828 - val_accuracy: 0.0190 - val_loss: 43.1794
Epoch 2/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 80s 37ms/step - accuracy: 0.9732 - loss: 0.0814 - val_accuracy: 0.0248 - val_loss: 43.4834
Epoch 3/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 80s 37ms/step - accuracy: 0.9877 - loss: 0.0404 - val_accuracy: 0.0310 - val_loss: 44.5475
Epoch 4/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 79s 36ms/step - accuracy: 0.9935 - loss: 0.0218 - val_accuracy: 0.0290 - val_loss: 46.2988
Epoch 5/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 79s 36ms/step - accuracy: 0.9915 - loss: 0.0281 - val_accuracy: 0.0333 - val_loss: 49.6415
Epoch 6/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 106s 49ms/step - accuracy: 0.9952 - loss: 0.0161 - val_accuracy: 0.0326 - val_loss: 43.7596
Epoch 7/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 77s 36ms/step - accuracy: 0.9967 - loss: 0.0132 - val_accuracy: 0.0326 - val_loss: 51.3461
Epoch 8/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 74s 34ms/step - accuracy: 

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split


X_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)


model = Sequential([
    LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 63)),  
    LSTM(128, return_sequences=True, activation='relu'),
    LSTM(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(29, activation='softmax')  
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val))
model.save("ai_sign_lstm_model.h5")

NameError: name 'x' is not defined